In [1]:
DATA_PATH = "fruits-360/"
TRAIN_PATH = DATA_PATH + "Training/"
TEST_PATH = DATA_PATH + "Test/"
NUMBER_OF_EPOCHS = 15

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision

In [3]:
torch.cuda.set_device(2)

In [4]:
def load_dataset(path, batch_size=5):
    train_dataset = torchvision.datasets.ImageFolder(
        root=path,
        transform=torchvision.transforms.ToTensor()
    )
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=batch_size,
        num_workers=0,
        shuffle=True
    )
    return train_dataset.class_to_idx, train_loader

In [5]:
train_class_to_id, train_set_loader = load_dataset(TRAIN_PATH)
test_class_to_id, test_set_loader = load_dataset(TEST_PATH, batch_size=5000)

In [6]:
number_of_classes = len(train_class_to_id)

In [7]:
class Custom_batch_norm(nn.Module):
    def __init__(self, num_features, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True):
        super(Custom_batch_norm, self).__init__()
        self.dimension = num_features
        self.eps = eps
        self.momentum = momentum
        self.affine = affine
        self.track_running_stats = track_running_stats
        if self.affine:
            self.gamma = nn.Parameter(torch.randn(self.dimension))
            self.beta = nn.Parameter(torch.zeros([self.dimension], dtype=torch.float32))
        
    def forward(self, x):
        a,b,c,d = x.shape
        x_transposed = x.transpose(0,1).contiguous().view(self.dimension, -1)
        mean = x_transposed.mean(1)
        variance = x_transposed.var(1)
        k = x_transposed - mean.view(self.dimension,1)
        n = variance + self.eps
        m = torch.sqrt(n)
        x_est = k / m.view(self.dimension, 1)
        
        return x_est.view(b,a,c,d).transpose(0,1) if self.affine is False else (x_est * self.gamma.view(self.dimension, 1) + self.beta.view(self.dimension, 1)).view(b,a,c,d).transpose(0,1)

In [8]:
class Fruit_conv_net(nn.Module):
    def __init__(self):
        super(Fruit_conv_net, self).__init__()
        self.conv_1_1 = nn.Conv2d(3, 15, 5, padding = 2, bias=False)
        self.bn_1 = Custom_batch_norm(15)
        nn.init.xavier_uniform_(self.conv_1_1.weight)
        self.pool_1 = nn.MaxPool2d(2)
        #self.dropout_1 = nn.Dropout2d(p=0.2)
        self.conv_1_2 = nn.Conv2d(15, 30, 5, padding = 2, bias=False)
        self.bn_2 = nn.BatchNorm2d(30)
        nn.init.xavier_uniform_(self.conv_1_2.weight)
        self.pool_2 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(30 * 25 * 25, 2000)
        nn.init.xavier_uniform_(self.fc1.weight)
        self.fc2 = nn.Linear(2000, 1000)
        nn.init.xavier_uniform_(self.fc2.weight)
        self.fc3 = nn.Linear(1000, number_of_classes)
        nn.init.xavier_uniform_(self.fc1.weight)

    def forward(self, x):
        x = F.relu(self.pool_1(self.bn_1(self.conv_1_1(x))))
        x = F.relu(self.pool_2(self.bn_2(self.conv_1_2(x))))
        x = x.view(-1, 30 * 25 * 25)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [9]:
network = Fruit_conv_net()
network.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(network.parameters(), lr=0.0001, weight_decay=1e-2)

In [10]:
for epoch in range(NUMBER_OF_EPOCHS):
    running_loss = 0.0
    for i, data in enumerate(train_set_loader, 0):
        # get the inputs
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = network(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.4f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_set_loader:
            images, labels = data
            images = images.cuda()
            labels = labels.cuda()
            outputs = network(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network:' + str(100 * correct / total))

print('Finished Training')

[1,  2000] loss: 1.1382
[1,  4000] loss: 0.2967
[1,  6000] loss: 0.2383
[1,  8000] loss: 0.2026


RuntimeError: CUDA out of memory. Tried to allocate 2.79 GiB (GPU 2; 11.91 GiB total capacity; 9.53 GiB already allocated; 1.69 GiB free; 128.31 MiB cached)